In [102]:
import joblib
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics


In [53]:
plt.style.use("ggplot")

# 导入数据
data =  pd.read_csv('./train.csv')
data_c = data.copy()
data.shape
data.head()

,TypeOfVisitor,UserPageVisitDuration,BounceRate,WebPageValue,BrowserServer,InformationRelatedPageVisitDuration,Month,ExitRate,ProductRelatedPages,ProductRelatedPageVisitDuration,DischargePattern,InformationRelatedPage,Weekend,Holiday,Area,OperatingSystem,UserRelatedPages,BuyOrNot
0,Returning_Visitor,136.00,0.000000,0.0,2,138.0,Mar,0.009697,54,2631.100000,2,3.0,False,0.0,1,1.0,2.0,0
1,Returning_Visitor,34.25,0.007310,0.0,2,68.5,Nov,0.013520,385,14505.727250,3,5.0,False,0.0,3,3.0,2.0,0
2,Returning_Visitor,0.00,0.013636,0.0,2,0.0,Dec,0.025515,111,1624.750000,2,0.0,False,0.0,1,2.0,0.0,0
3,Returning_Visitor,0.00,0.007018,0.0,2,0.0,Nov,0.027875,57,1091.005876,2,0.0,False,0.0,6,2.0,0.0,0
4,Returning_Visitor,0.00,0.200000,0.0,4,0.0,June,0.200000,1,0.000000,3,0.0,False,0.0,9,2.0,0.0,0


In [66]:
category_features=['TypeOfVisitor','Month','Weekend']
numerical_features=['UserPageVisitDuration','BounceRate ','WebPageValue','BrowserServer','InformationRelatedPageVisitDuration ','ExitRate','ProductRelatedPages ','ProductRelatedPageVisitDuration','DischargePattern','InformationRelatedPage','Holiday','Area','OperatingSystem','UserRelatedPages']
y_feature='BuyOrNot'

In [70]:
data[category_features]=data[category_features].astype('category')

In [71]:
#data[category_features].head()
#类别型特征

In [72]:
data['BuyOrNot'].value_counts()
# 1为购买，0为未购买

BuyOrNot
0    7295
1    1336
Name: count, dtype: int64

In [73]:
features=[]
cols=data.columns.tolist()
for each in cols:
    if each != 'BuyOrNot':
        features.append(each)

In [74]:
len(features)

17

In [75]:
train_data=data[features]
train_target=data['BuyOrNot']

In [76]:
X_train,X_test,y_train,y_test=train_test_split(train_data,train_target,test_size=0.2,random_state=0)

In [77]:
X_test.shape

(1727, 17)

In [78]:
X_train.shape

(6904, 17)

In [79]:
#LGB格式
lgb_train=lgb.Dataset(X_train,y_train)
lgb_eval=lgb.Dataset(X_test,y_test,reference=lgb_train)

In [82]:
params={"num-leaves":60,
        "objective":"binary",
        "boosting":"gbdt",
        "metric":{"binary_logloss","auc"}
        }

In [81]:
clf=lgb.train(params,lgb_train,num_boost_round=200,valid_sets=lgb_eval,early_stopping_rounds=60,feature_name=features)

[LightGBM] [Warning] Unknown parameter: num-leaves
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: num-leaves
[LightGBM] [Info] Number of positive: 1051, number of negative: 5853
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000427 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1887
[LightGBM] [Info] Number of data points in the train set: 6904, number of used features: 17
[LightGBM] [Warning] Unknown parameter: num-leaves
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.152231 -> initscore=-1.717212
[LightGBM] [Info] Start training from score -1.717212
[1]	valid_0's auc: 0.902627	valid_0's binary_logloss: 0.401782
Training until validation scores don't improve for 60 rounds
[2]	v

C:\Users\h\anaconda3\envs\py_dataAnalysis\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\h\anaconda3\envs\py_dataAnalysis\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\h\anaconda3\envs\py_dataAnalysis\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


In [89]:
probs=clf.predict(X_test,num_iteration=clf.best_iteration)

In [90]:
probs

array([0.01583974, 0.21036819, 0.01169468, ..., 0.10050444, 0.04144126,
       0.01253157])

In [91]:
preds=[0 if each<0.5 else 1 for each in probs]

In [94]:
count=0
for i in range(len(preds)):
    if preds[i]==y_test.iloc[i]:
        count+=1
print('准确率是{}'.format(count/len(preds)))

准确率是0.9044585987261147


In [97]:
metrics.roc_auc_score(y_test,probs)

0.9347470618293305

In [98]:
clf.save_model('./model.txt')

In [103]:
joblib.dump(filename='clf.model',value=clf)

['clf.model']

In [104]:
model1=joblib.load(filename="clf.model")

In [111]:
data1=pd.DataFrame(data=preds)

data1.to_csv('./test.csv',index=False) # 如果生成excel，可以用to_exce